In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [11]:
ticker = input("Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA): ").strip().upper()
data = yf.download(ticker, period="1y")  # Fetch stock data
print(data.head())  # Debugging: Check if data is fetched

Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA):  AAPL


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2024-03-20  177.830307  177.830307  174.267130  174.894174   53423100
2024-03-21  170.564621  176.655870  170.037113  176.217935  106181300
2024-03-22  171.470352  172.236738  169.260784  170.952792   71106600
2024-03-25  170.047073  171.131947  168.653644  169.768390   54288300
2024-03-26  168.912430  170.614385  168.783036  169.201060   57388400


In [12]:
def get_rsi(data, period=14):
    """

    Calculates the Relative Strength Index (RSI) for a given stock data.
    """
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Prevent division by zero
    rs = avg_gain / avg_loss.replace(0, np.nan)  # Replace 0 with NaN to avoid division by zero
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


In [13]:
def get_macd(data, short_window=12, long_window=26, signal_window=9):

    short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
    macd = short_ema - long_ema
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    histogram = macd - signal
    return macd, signal, histogram

In [14]:
def get_bollinger_bands(data, window=20, num_std=2):
   
    rolling_mean = data['Close'].rolling(window=window).mean()
    rolling_std = data['Close'].rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    bandwidth = (upper_band - lower_band) / rolling_mean
    return upper_band, lower_band, bandwidth


In [15]:
def get_momentum(data, period=14):
 
    return data['Close'].diff(period)


In [16]:
def get_volatility(data, period=14):
  
    return data['Close'].pct_change().rolling(window=period).std()


In [17]:
def get_stock_data(ticker):
    
    data = yf.download(ticker, period="1y")  # Download 1 year of data
    return data

In [18]:
def prepare_data(data):
    """
    Prepares the data for machine learning by adding features and labels.
    """
    
    # Calculate price changes FIRST
    data['Price_Change'] = data['Close'].pct_change()

    # Add RSI
    data['RSI'] = get_rsi(data)

    # Check if RSI was created successfully (case insensitive)
    if 'RSI' not in data.columns:
        raise ValueError("RSI column not created. Check get_rsi() function.")

    # Add Moving Averages
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()

    # Add MACD
    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    # Add Bollinger Bands
    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    # Add Momentum
    data['Momentum'] = get_momentum(data)

    # Add Volatility
    data['Volatility'] = get_volatility(data)

   # Add Volume Change
       # Add Volume Change
    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    # Define buy/sell signals with NaN handling
    buy_condition = (data['RSI'] < 30) & (data['Price_Change'] > 0)
    buy_condition = buy_condition.fillna(False)  # Fix: Assign the result back to buy_condition
    sell_condition = (data['RSI'] > 70) & (data['Price_Change'] < 0)
    sell_condition = sell_condition.fillna(False)  # Fix: Assign the result back to sell_condition

    # Set the Signal column based on conditions
    data['Signal'] = 0
    data.loc[buy_condition, 'Signal'] = 1  # Buy signal
    data.loc[sell_condition, 'Signal'] = -1  # Sell signal

    return data

In [19]:
def prepare_data(data):
    """
    Prepares the data for machine learning by adding features and labels.
    """
    # Calculate price changes FIRST
    data['Price_Change'] = data['Close'].pct_change()

    # Add RSI
    data['RSI'] = get_rsi(data)

    # Check if RSI was created successfully (case insensitive)
    if 'RSI' not in data.columns:
        raise ValueError("RSI column not created. Check get_rsi() function.")

    # Add Moving Averages
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()

    # Add MACD
    macd, signal, histogram = get_macd(data)
    data['MACD'] = macd
    data['MACD_Signal'] = signal
    data['MACD_Histogram'] = histogram

    # Add Bollinger Bands
    upper_band, lower_band, bandwidth = get_bollinger_bands(data)
    data['Bollinger_Upper'] = upper_band
    data['Bollinger_Lower'] = lower_band
    data['Bollinger_Bandwidth'] = bandwidth

    # Add Momentum
    data['Momentum'] = get_momentum(data)

    # Add Volatility
    data['Volatility'] = get_volatility(data)

    # Add Volume Change
    data['Volume_Change'] = data['Volume'].pct_change().fillna(0)

    # Define buy/sell signals with NaN handling
    buy_condition = (data['RSI'] < 30) & (data['Price_Change'] > 0)
    buy_condition = buy_condition.fillna(False)  # Fix: Assign the result back to buy_condition
    sell_condition = (data['RSI'] > 70) & (data['Price_Change'] < 0)
    sell_condition = sell_condition.fillna(False)  # Fix: Assign the result back to sell_condition

    # Set the Signal column based on conditions
    data['Signal'] = 0
    data.loc[buy_condition, 'Signal'] = 1  # Buy signal
    data.loc[sell_condition, 'Signal'] = -1  # Sell signal

    # Drop rows with NaN values (due to feature calculations)
    data.dropna(inplace=True)
    return data

In [23]:
def train_model(data):
    # Define feature columns
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    
    # Ensure features exist in the data
    missing_cols = [col for col in features if col not in data.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns in data: {missing_cols}")

    # Extract features and target variable
    X = data[features]
    y = data.get('Signal')  # Avoid KeyError

    if y is None:
        raise ValueError("Target variable 'Signal' is missing from the dataset.")

    # Convert to DataFrame to avoid warnings
    y = y.to_frame()

    # Handle NaN values: Fill with column mean
    imputer = SimpleImputer(strategy="mean")
    X = pd.DataFrame(imputer.fit_transform(X), columns=features, index=data.index)

    # Remove NaNs from y
    y.dropna(inplace=True)

    # Ensure X and y stay aligned
    X = X.loc[y.index]

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Standardize features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train the model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train.values.ravel())  # Convert y_train to a 1D array

    return model

In [24]:
def generate_alerts(data, model):
    # Features
    features = ['RSI', 'SMA_10', 'SMA_50', 'MACD', 'MACD_Signal', 'MACD_Histogram',
                'Bollinger_Upper', 'Bollinger_Lower', 'Bollinger_Bandwidth',
                'Momentum', 'Volatility', 'Volume_Change']
    
    # Predict signals using the trained model
    data['Predicted_Signal'] = model.predict(data[features])

    # Generate alerts
    alerts = []
    for i in range(len(data)):  # <-- Fixed indentation here
        if data['Predicted_Signal'].iloc[i] == 1:
            alerts.append(("Buy", data.index[i], data['Close'].iloc[i]))
        elif data['Predicted_Signal'].iloc[i] == -1:
            alerts.append(("Sell", data.index[i], data['Close'].iloc[i]))
        else:
            alerts.append(("Hold", data.index[i], data['Close'].iloc[i]))

    return alerts


In [25]:
def analyze_stock(ticker):
    """
    Analyzes a stock and provides buy/sell/hold alerts.
    """
    # Fetch and prepare data
    data = get_stock_data(ticker)
    data = prepare_data(data)

    # Train the model
    model = train_model(data)

    # Generate alerts
    alerts = generate_alerts(data, model)

    # Print alerts
    print(f"\nAlerts for {ticker}:")
    for alert in alerts:
        print(f"{alert[0]} Signal at {alert[1]}: Price: {alert[2]}")

if __name__ == "__main__":
    # Input stock ticker from the user
    ticker = input("Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA): ").strip().upper()

    # Analyze the stock
    analyze_stock(ticker)

Enter the stock ticker symbol (e.g., AAPL, MSFT, TSLA):  AAPL


[*********************100%***********************]  1 of 1 completed


Alerts for AAPL:
Hold Signal at 2024-05-30 00:00:00: Price: Ticker
AAPL    190.649246
Name: 2024-05-30 00:00:00, dtype: float64
Hold Signal at 2024-05-31 00:00:00: Price: Ticker
AAPL    191.606033
Name: 2024-05-31 00:00:00, dtype: float64
Hold Signal at 2024-06-03 00:00:00: Price: Ticker
AAPL    193.380066
Name: 2024-06-03 00:00:00, dtype: float64
Hold Signal at 2024-06-04 00:00:00: Price: Ticker
AAPL    193.699005
Name: 2024-06-04 00:00:00, dtype: float64
Hold Signal at 2024-06-05 00:00:00: Price: Ticker
AAPL    195.213898
Name: 2024-06-05 00:00:00, dtype: float64
Hold Signal at 2024-06-06 00:00:00: Price: Ticker
AAPL    193.828552
Name: 2024-06-06 00:00:00, dtype: float64
Hold Signal at 2024-06-07 00:00:00: Price: Ticker
AAPL    196.230484
Name: 2024-06-07 00:00:00, dtype: float64
Hold Signal at 2024-06-10 00:00:00: Price: Ticker
AAPL    192.473114
Name: 2024-06-10 00:00:00, dtype: float64
Hold Signal at 2024-06-11 00:00:00: Price: Ticker
AAPL    206.456116
Name: 2024-06-11 00:00:00